In [44]:
!ls "gpt_results"

compute_dependence_with_base_changes=False,sequence_completion_equality.model=davinci,string_transformation_completion_equality=False
compute_dependence_with_base_changes=False,sequence_completion_equality.model=gpt-3.5-turbo,string_transformation_completion_equality=False
compute_dependence_with_base_changes=False,sequence_completion_equality.model=gpt-4-0314,string_transformation_completion_equality=False
compute_dependence_with_base_changes=False,sequence_completion_equality.model=text-davinci-003,string_transformation_completion_equality=False
multirun.yaml


In [62]:
import numpy as np

models = [
    "davinci", "gpt-3.5-turbo", "gpt-4-0314", "text-davinci-003"
]
results = []
for answer_validity in ['valid', 'all']:
    for model in models:
        df = pd.read_csv(
                f'./gpt_results/compute_dependence_with_base_changes=False,sequence_completion_equality.model={model},string_transformation_completion_equality=False/evaluate_sequence_completion_equality/sequence_completion_equality_evaluation_{model}.csv'
        )
        match_accs, model_match_accs, model_consistency_accs, consistent_and_matched = (
            [],
            [],
            [],
            [],
        )
        for i, data in df.iterrows():
            if answer_validity == 'valid' and  data["generated_completion_matches"] is not True and data["generated_completion_matches"] is not False:
                continue
            match_accs.append(1 if data["generated_completion_matches"] == True else 0)

        for i, data in df.iterrows():
            if answer_validity == 'valid' and data["model_completion_matches"] is not True and data["model_completion_matches"] is not False:
                continue
            model_match_accs.append(1 if data["model_completion_matches"] == True else 0)
            
        for i, data in df.iterrows():
            if answer_validity == 'valid' and data["model_self_consistency_evaluation"].strip() != "Y" and data["model_self_consistency_evaluation"].strip() != "N":
                continue
            model_consistency_accs.append(
                1 if data["model_self_consistency_evaluation"].strip() == "Y" else 0
            )
            
        for i, data in df.iterrows():
            if answer_validity == 'valid' and data["model_self_consistency_evaluation"].strip() != "Y" and data["model_self_consistency_evaluation"].strip() != "N":
                continue
            if answer_validity == 'valid' and  data["generated_completion_matches"] is not True and data["generated_completion_matches"] is not False:
                continue
            consistent_and_matched.append(
                1
                if data["model_self_consistency_evaluation"].strip() == "Y"
                and data["generated_completion_matches"]
                else 0
            )


        ground_truth_consistent = round(np.mean(match_accs), 4) * 100
        self_rule_following_consistency = round(np.mean(model_match_accs), 4) * 100
        self_comparison_consistency = round(np.mean(model_consistency_accs), 4) * 100
        consistent_and_matched_accuracy = round(np.mean(consistent_and_matched), 4) * 100
        total_sequences = 225
        
        results.append({
            "model": model,
            "answer_validity": answer_validity,
            "ground_truth_consistent": ground_truth_consistent,
            "ground_truth_consistent_num": len(match_accs),
            "self_rule_following_consistency":  self_rule_following_consistency, 
            "self_rule_following_consistency_len": len(model_match_accs),
            "self_comparison_consistency": self_comparison_consistency,
            "self_comparison_consistency_len": len(model_consistency_accs),
            "consistent_and_matched_accuracy": consistent_and_matched_accuracy,
            "consistent_and_matched_accuracy_len": len(consistent_and_matched)
        })
        print(
            f"""
            For {model} including {answer_validity} answers
            Evaluated {count} ambiguous sequences of {total_sequences} total.
            Resulting in:
            - {ground_truth_consistent}% ground-truth-consistent (using {len(match_accs)})
            - {self_rule_following_consistency}% self-rule-following-consistency (using {len(model_match_accs)})
            - {self_comparison_consistency}% self-comparison-consistency (using {len(model_consistency_accs)})
            - {consistent_and_matched_accuracy}% self-comparison-consistency and ground-truth-consistent. (using {len(consistent_and_matched)})
            """
    )
pd.DataFrame(results).to_csv('./q0_results.csv')


            For davinci including valid answers
            Evaluated 173 ambiguous sequences of 225 total.
            Resulting in:
            - 26.090000000000003% ground-truth-consistent (using 23)
            - 56.52% self-rule-following-consistency (using 23)
            - 96.31% self-comparison-consistency (using 217)
            - 26.090000000000003% self-comparison-consistency and ground-truth-consistent. (using 23)
            

            For gpt-3.5-turbo including valid answers
            Evaluated 173 ambiguous sequences of 225 total.
            Resulting in:
            - 94.67999999999999% ground-truth-consistent (using 188)
            - 95.74000000000001% self-rule-following-consistency (using 188)
            - 94.42% self-comparison-consistency (using 215)
            - 89.89% self-comparison-consistency and ground-truth-consistent. (using 188)
            

            For gpt-4-0314 including valid answers
            Evaluated 173 ambiguous sequences of 225 

In [61]:
consistent_functions = []
for model in models:
    df = pd.read_csv(
            f'./gpt_results/compute_dependence_with_base_changes=False,sequence_completion_equality.model={model},string_transformation_completion_equality=False/evaluate_sequence_completion_equality/sequence_completion_equality_evaluation_{model}.csv'
    )
    for i, data in df.iterrows():
        if data["generated_completion_matches"] is True:
            consistent_functions.append({
                "fn": data['original_function'],
                "model": model
            })
            
pd.DataFrame(
    consistent_functions
).to_csv('./consistent_functions_by_model.csv')
            
    